In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme()

In [ ]:
log_file = "../../remote/qiskit-qulacs-comparison-amd/qlbm.log"
with open(log_file, "r") as f:
    lines = f.readlines()

session_line = [c for c, line in enumerate(lines) if "Session" in line][1]

lines_statevector_true = lines[:session_line]
lines_statevector_false = lines[session_line:]

In [ ]:
# Process statevector=True lines
combination_lines_indices = [
    c for c, line in enumerate(lines_statevector_true) if "Combination #" in line
]

sections = [
    lines_statevector_true[
        combination_lines_indices[c] : combination_lines_indices[c + 1]
        if c < len(combination_lines_indices)
        else session_line
    ]
    for c in range(len(combination_lines_indices) - 1)
]

statevector_true_records = []
for section in sections:
    sec_info = section[1].split("INFO: ")[-1].rstrip().split(", ")
    time_elapsed_ns = section[-1].split("INFO: ")[-1].rstrip().split()[-2]
    step_simulation_line_indices = [
        c for c, line in enumerate(section) if "Main circuit for step" in line
    ]
    qiskit_start = [
        c for c, line in enumerate(section) if "Executing QISKIT..." in line
    ][0]
    total_duration = 0
    for sl in step_simulation_line_indices:
        step_number = section[sl].split("for step ")[-1].split()[0]
        props = section[sl].split("INFO: ")[-1].rstrip().split(", ")[1:]
        duration = section[sl + 1].split()[-2]
        total_duration += int(duration)

        statevector_true_records.append(
            {
                "Lattice": sec_info[0].split("=")[-1].split("/")[-1].split(".")[0],
                "Dimensions": sec_info[0]
                .split("=")[-1]
                .split("/")[-1]
                .split(".")[0]
                .split("_")[1],
                "Obstacles": sec_info[0]
                .split("=")[-1]
                .split("/")[-1]
                .split(".")[0]
                .split("_")[2],
                "Circuit Qubits": int(sec_info[1].split("=")[-1]),
                "Step": int(step_number),
                "Depth": int(props[1]),
                "Gates": int(props[-1][:-1]),
                "Duration (ns)": int(duration),
                "Cumulative Duration (ns)": int(total_duration),
                "Snapshots": True,
                "Platform": "QISKIT"
                if sl > qiskit_start
                else "QULACS",  # Qulacs is first in this simulation
            }
        )

sv_true_df = pd.DataFrame.from_records(statevector_true_records)
sv_true_df

In [ ]:
# Process statevector=False lines
combination_lines_indices = [
    c for c, line in enumerate(lines_statevector_false) if "Combination #" in line
]

sections = [
    lines_statevector_false[
        combination_lines_indices[c] : combination_lines_indices[c + 1]
        if c < len(combination_lines_indices)
        else -1
    ]
    for c in range(len(combination_lines_indices) - 1)
]

statevector_false_records = []
for section in sections:
    sec_info = section[1].split("INFO: ")[-1].rstrip().split(", ")
    time_elapsed_ns = section[-1].split("INFO: ")[-1].rstrip().split()[-2]
    step_simulation_line_indices = [
        c for c, line in enumerate(section) if "Main circuit for step" in line
    ]
    qiskit_start = [
        c for c, line in enumerate(section) if "Executing QISKIT..." in line
    ][0]
    total_duration = 0
    for sl in step_simulation_line_indices:
        step_number = section[sl].split("for step ")[-1].split()[0]
        props = section[sl].split("INFO: ")[-1].rstrip().split(", ")[1:]
        duration = section[sl + 1].split()[-2]

        total_duration += int(duration)
        statevector_false_records.append(
            {
                "Lattice": sec_info[0].split("=")[-1].split("/")[-1].split(".")[0],
                "Dimensions": sec_info[0]
                .split("=")[-1]
                .split("/")[-1]
                .split(".")[0]
                .split("_")[1],
                "Obstacles": sec_info[0]
                .split("=")[-1]
                .split("/")[-1]
                .split(".")[0]
                .split("_")[2],
                "Circuit Qubits": int(sec_info[1].split("=")[-1]),
                "Step": int(step_number),
                "Depth": int(props[1]),
                "Gates": int(props[-1][:-1]),
                "Duration (ns)": int(duration),
                "Cumulative Duration (ns)": int(total_duration),
                "Snapshots": False,
                "Platform": "QISKIT"
                if sl > qiskit_start
                else "QULACS",  # Qulacs is first in this simulation
            }
        )

sv_false_df = pd.DataFrame.from_records(statevector_false_records)
sv_false_df

In [ ]:
cdf = pd.concat([sv_true_df])
cdf["Duration (s)"] = cdf["Duration (ns)"] / 1e9
cdf["Cumulative Duration (s)"] = cdf["Cumulative Duration (ns)"] / 1e9


In [ ]:
pdf = cdf[cdf["Dimensions"].isin(["32x32"])]
# pdf = pdf[pdf["Obstacles"] == 1]
sns.lineplot(
    pdf,
    x="Step",
    y="Cumulative Duration (s)",
    hue="Obstacles",
    style="Platform",
    markers=True,
)
plt.savefig("bad_qulacs_plot.pdf", format="pdf")